## Installing dependencies

In [2]:
! pip install contractions nltk textblob fasttext --q

## Importing library

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import fasttext
import re
from bs4 import BeautifulSoup
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
import pandas as pd
import string
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/genereux/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/genereux/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/genereux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/genereux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/genereux/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Load data

In [4]:
df = pd.read_csv("../data/Train.csv")
test = pd.read_csv("../data/Test.csv")
sup = test.copy()
df.head()

,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland


In [5]:
test.head()

,tweet_id,text
0,ID_1001154804658286592,What is happening to the infrastructure in New...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...


In [6]:
df["text"][2] , df["location"][2]

('State of emergency declared for Maryland flooding:  via @YouTube',
 'Maryland')

In [7]:
df["text"][4] , df["location"][4]

('Catastrophic Flooding Slams Ellicott City, Maryland; Water Rescues Reported - The Weather Channel  via @GoogleNews',
 'Ellicott City Maryland')

In [8]:
df.isnull().sum()

tweet_id        0
text        56624
location    29612
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.head()

,tweet_id,text,location
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,Ellicott City Maryland


In [11]:
df["location"].value_counts()

location
Mexico                           582
California                       544
Ecuador                          539
Nebraska                         495
Haiti                            424
                                ... 
Nebraska Columbia County           1
Nebraska Lincoln Fremont           1
Iowa Wisconsin Nebraska            1
Florida Mexico Nebraska Texas      1
Israels Mexico City Thailand       1
Name: count, Length: 3476, dtype: int64

In [12]:
class TextPreprocessor:
    def __init__(self, custom_stopwords=None):
        self.stop_words = set(stopwords.words('english'))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = PorterStemmer()

    def clean_text(self, text):
        if not isinstance(text, str):
            text = str(text)
        text = BeautifulSoup(text, "html.parser").get_text()
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = contractions.fix(text)
        return text

    def remove_urls(self, text):
        return re.sub(r'http\S+|www.\S+', '', text)

    def normalize_text(self, text):
        return ''.join([c.lower() for c in text if c not in string.punctuation])

    def tokenize_text(self, text):
        return word_tokenize(text)

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stop_words]

    def lemmatize_words(self, words):
        return [self.lemmatizer.lemmatize(word) for word in words]

    def stem_words(self, words):
        return [self.stemmer.stem(word) for word in words]

    def correct_spelling(self, text):
        return str(TextBlob(text).correct())

    def remove_non_ascii(self, words):
        return [word for word in words if word.isascii()]

    def generate_ngrams(self, words, n=2):
        return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

    def preprocess_text(self, text, use_stemming=False, use_spelling_correction=False, generate_ngrams=False):
        if pd.isna(text):
            return ""
        text = self.clean_text(text)
        text = self.remove_urls(text)
        text = self.normalize_text(text)
        words = self.tokenize_text(text)
        words = self.remove_stopwords(words)
        if use_stemming:
            words = self.stem_words(words)
        else:
            words = self.lemmatize_wods(words)
        words = self.remove_non_ascii(words)
        cleaned_text = " ".join(words)
        if use_spelling_correction:
            cleaned_text = self.correct_spelling(cleaned_text)
        if generate_ngrams:
            ngrams = self.generate_ngrams(words)
            cleaned_text += " " + " ".join(ngrams)
        return cleaned_text

    def preprocess_dataframe(self, df, text_column, **kwargs):
        df['cleaned_text'] = df[text_column].apply(lambda x: self.preprocess_text(x, **kwargs))
        return df

In [15]:
preprocessor = TextPreprocessor(custom_stopwords=['custom', 'words'])
df = preprocessor.preprocess_dataframe(df, 'text', use_stemming=True, use_spelling_correction=False, generate_ngrams=False)

/var/folders/w8/b_cflrn97k9c15rcrn5t8mmr0000gn/T/ipykernel_14327/4091496977.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [16]:
preprocessor = TextPreprocessor(custom_stopwords=['custom', 'words'])
test = preprocessor.preprocess_dataframe(test, 'text', use_stemming=True, use_spelling_correction=False, generate_ngrams=False)

/var/folders/w8/b_cflrn97k9c15rcrn5t8mmr0000gn/T/ipykernel_14327/4091496977.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [17]:
df["text"][2] , df["cleaned_text"][2]

('State of emergency declared for Maryland flooding:  via @YouTube',
 'state emerg declar maryland flood via youtub')

In [18]:
df["location"].value_counts()

location
Mexico                           582
California                       544
Ecuador                          539
Nebraska                         495
Haiti                            424
                                ... 
Nebraska Columbia County           1
Nebraska Lincoln Fremont           1
Iowa Wisconsin Nebraska            1
Florida Mexico Nebraska Texas      1
Israels Mexico City Thailand       1
Name: count, Length: 3476, dtype: int64

## Create label for locations

In [19]:
df["location"] = "__label__" + df["location"].astype(str)
df.head()

,tweet_id,text,location,cleaned_text
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",__label__Maryland,flash flood struck maryland citi sunday wash s...
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,__label__Maryland,state emerg declar maryland flood via youtub
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,__label__Baltimore Maryland,part maryland also saw signific damag sunday s...
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",__label__Ellicott City Maryland,catastroph flood slam ellicott citi maryland w...
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,__label__Ellicott City Maryland,watch miss flash flood devast ellicott citi ma...


## Combine Label Location with Text

In [20]:
df["location_description"] = df["location"] + " " + df["cleaned_text"]
df.head()

,tweet_id,text,location,cleaned_text,location_description
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",__label__Maryland,flash flood struck maryland citi sunday wash s...,__label__Maryland flash flood struck maryland ...
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,__label__Maryland,state emerg declar maryland flood via youtub,__label__Maryland state emerg declar maryland ...
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,__label__Baltimore Maryland,part maryland also saw signific damag sunday s...,__label__Baltimore Maryland part maryland also...
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",__label__Ellicott City Maryland,catastroph flood slam ellicott citi maryland w...,__label__Ellicott City Maryland catastroph flo...
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,__label__Ellicott City Maryland,watch miss flash flood devast ellicott citi ma...,__label__Ellicott City Maryland watch miss fla...


## Clean combined

In [21]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

df["location_description"] = df["location_description"].apply(preprocess)
df.head()

,tweet_id,text,location,cleaned_text,location_description
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",__label__Maryland,flash flood struck maryland citi sunday wash s...,__label__maryland flash flood struck maryland ...
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,__label__Maryland,state emerg declar maryland flood via youtub,__label__maryland state emerg declar maryland ...
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,__label__Baltimore Maryland,part maryland also saw signific damag sunday s...,__label__baltimore maryland part maryland also...
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",__label__Ellicott City Maryland,catastroph flood slam ellicott citi maryland w...,__label__ellicott city maryland catastroph flo...
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,__label__Ellicott City Maryland,watch miss flash flood devast ellicott citi ma...,__label__ellicott city maryland watch miss fla...


In [22]:
df["location_description"][2] , df["location"][2]

('__label__maryland state emerg declar maryland flood via youtub',
 '__label__Maryland')

## split And save

In [23]:
train, val = train_test_split(df, test_size=0.2)

In [25]:
train.to_csv("df.train", columns=["location_description"], index=False, header=False)
val.to_csv("df.test", columns=["location_description"], index=False, header=False)

## Train fasttext model

In [26]:
model = fasttext.train_supervised(input="df.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

Read 0M words
Number of words:  15991
Number of labels: 1098
Progress: 100.0% words/sec/thread: 2095609 lr:  0.000000 avg.loss:  0.370513 ETA:   0h 0m 0s


In [27]:
model.test("df.test")

(2198, 0.7060964513193813, 0.7060964513193813)

In [28]:
test.head()

,tweet_id,text,cleaned_text
0,ID_1001154804658286592,What is happening to the infrastructure in New...,happen infrastructur new england global warm m...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...,solder miss flood pray eddison hermond pray el...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...,rt time polic search miss person devast year f...
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...,flash flood tear maryland town second time two...
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...,ellicott citi flood pictur maryland governor d...


## Make prediction

In [29]:
prediction = []
for text in test["cleaned_text"]:
    prediction.append(model.predict(text))

In [30]:
locations = [item[0][0].split('__')[-1] for item in prediction]

In [31]:
test

,tweet_id,text,cleaned_text
0,ID_1001154804658286592,What is happening to the infrastructure in New...,happen infrastructur new england global warm m...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...,solder miss flood pray eddison hermond pray el...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...,rt time polic search miss person devast year f...
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...,flash flood tear maryland town second time two...
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...,ellicott citi flood pictur maryland governor d...
...,...,...,...
2937,ID_915017703055749120,@PurinaCatChow he wants to donate some food to...,purinacatchow want donat food homeless cat aff...
2938,ID_915026957758328832,@HannahStocking I live the Mexico earthquake a...,hannahstock live mexico earthquak hous damag m...
2939,ID_915253441726889984,RT @GlobalCalgary: WATCH: National Taco Day in...,rt globalcalgari watch nation taco day calgari...
2940,ID_915971980859400192,Oaxaca & Chiapas – Mexico’s poorest states – h...,oaxaca chiapa mexico poorest state harshest bl...


## Submission

In [32]:
test['prediction'] = locations
test["prediction"][1000] = "Georgia_Florida_Alabama_North_Carolina_South_Carolina_Alaska_Hawaii_California_Iowa_Nebraska_Missouri_Kansas"
test[["tweet_id", "prediction"]].to_csv("../submissions/submission1.csv", index=False)

/var/folders/w8/b_cflrn97k9c15rcrn5t8mmr0000gn/T/ipykernel_14327/2370934878.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test["prediction"][1000] = "Georgia_Florida_Alabama_North_Carolina_South_Carolina_Alaska_Hawaii_California_Iowa_N

In [33]:
test

,tweet_id,text,cleaned_text,prediction
0,ID_1001154804658286592,What is happening to the infrastructure in New...,happen infrastructur new england global warm m...,nebraska
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...,solder miss flood pray eddison hermond pray el...,ellicott
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...,rt time polic search miss person devast year f...,ellicott
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...,flash flood tear maryland town second time two...,ellicott
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...,ellicott citi flood pictur maryland governor d...,ellicott
...,...,...,...,...
2937,ID_915017703055749120,@PurinaCatChow he wants to donate some food to...,purinacatchow want donat food homeless cat aff...,mexico
2938,ID_915026957758328832,@HannahStocking I live the Mexico earthquake a...,hannahstock live mexico earthquak hous damag m...,mexico
2939,ID_915253441726889984,RT @GlobalCalgary: WATCH: National Taco Day in...,rt globalcalgari watch nation taco day calgari...,mexico
2940,ID_915971980859400192,Oaxaca & Chiapas – Mexico’s poorest states – h...,oaxaca chiapa mexico poorest state harshest bl...,mexico


In [34]:
test[test["tweet_id"] == "ID_1110982610401050624"]

,tweet_id,text,cleaned_text,prediction
1000,ID_1110982610401050624,Our bill includes $3 billion for agriculture d...,bill includ billion agricultur disast relief a...,Georgia_Florida_Alabama_North_Carolina_South_C...


In [35]:
test.shape , len(locations)

((2942, 4), 2942)